<a href="https://colab.research.google.com/github/PersGraphics/collage/blob/main/scrapy_tinosoriano_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del proyecto

In [ ]:
# instalación de Scrapy
!pip install Scrapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self

In [ ]:
# creación del proyecto
!scrapy startproject project_faces scrapy

New Scrapy project 'project_faces', using template directory '/usr/local/lib/python3.8/dist-packages/scrapy/templates/project', created in:
    /content/scrapy

You can start your first spider with:
    cd scrapy
    scrapy genspider example example.com


In [ ]:
# creación del spider
!cd scrapy/project_faces && scrapy genspider FindFaces https://www.tinosoriano.com/blog/

Created spider 'FindFaces' using template 'basic' in module:
  project_faces.spiders.FindFaces


## Edición de path/to/settings.py



*   USER_AGENT
*   ROBOTSTXT_OBEY
*   DEFAULT_REQUEST_HEADERS



In [ ]:
%%writefile scrapy/project_faces/settings.py

BOT_NAME = 'project_faces'

SPIDER_MODULES = ['project_faces.spiders']
NEWSPIDER_MODULE = 'project_faces.spiders'

USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'

ROBOTSTXT_OBEY = False

DEFAULT_REQUEST_HEADERS = {
   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
   'Accept-Language': 'es',
}

REQUEST_FINGERPRINTER_IMPLEMENTATION = '2.7'
TWISTED_REACTOR = 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'

Overwriting scrapy/project_faces/settings.py


In [97]:
%%writefile scrapy/project_faces/spiders/FindFaces.py
import scrapy

class FindfacesSpider(scrapy.Spider):
    name = 'FindFaces'
    allowed_domains = ['www.tinosoriano.com']
    start_urls = ['https://www.tinosoriano.com/blog/']
    pages = 1
    max_web = 1
    def parse(self, response):
        posts = response.css('.entry-header')
        for post in posts:
            href = post.css('.entry-title > a::attr(href)').get()
            print("href:", href)
            yield scrapy.Request(href, callback = self.parse_post, meta={'href': href})
        next_page = response.css('.pagination-next')
        print(next_page)
        self.pages += 1
        if (len(next_page) > 0) and (self.pages < self.max_web):
            next_href = next_page.css('a::attr(href)').get()
            yield scrapy.Request(next_href)

    def parse_post(self, response):
        href = response.meta.get('href')    
        element = response.xpath('//figure[@class="wp-block-image"]')
        for item in element:
            img = item.css('img::attr(src)').get()
            yield { 'img': img ,'href': href}

Overwriting scrapy/project_faces/spiders/FindFaces.py


# Ejecución del spider

In [105]:
!cd scrapy/project_faces && scrapy crawl FindFaces -o faces.json

2022-12-14 18:19:07 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: project_faces)
2022-12-14 18:19:07 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.8.16 (default, Dec  7 2022, 01:12:13) - [GCC 7.5.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.4, Platform Linux-5.10.133+-x86_64-with-glibc2.27
2022-12-14 18:19:07 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'project_faces',
 'NEWSPIDER_MODULE': 'project_faces.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'SPIDER_MODULES': ['project_faces.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor',
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'}
2022-12-14 18:19:07 [asyncio] DEBUG: Using selector: EpollSelector
2022-12-14 18:19:07 [scrapy.utils.log] DEBUG: Using reactor: twis

# Implementación del ejercicio del collage

In [103]:
from io import BytesIO
import cv2
from PIL import Image
import json
import numpy as np
import requests

In [106]:
json_file = 'scrapy/project_faces/faces.json'
with open(json_file, 'r') as f:
  data = json.load(f)

images = []
for item in data:
  url = item["img"]
  img = Image.open(BytesIO(requests.get(url).content))
  img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
  images.append(img)

In [110]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
faces_coords = []
face_count = 0
for img in images:
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor = 1.2,
        minNeighbors = 8,
        minSize = (30, 30)
  )
  faces_coords.append(faces)
  print("Found {0} Faces!".format(len(faces)))
  face_count += len(faces)

Found 0 Faces!
Found 0 Faces!
Found 0 Faces!
Found 2 Faces!
Found 1 Faces!
Found 1 Faces!
Found 1 Faces!
Found 0 Faces!
Found 0 Faces!
Found 2 Faces!
Found 3 Faces!
Found 0 Faces!
Found 1 Faces!
Found 1 Faces!
Found 0 Faces!
Found 1 Faces!
Found 0 Faces!
Found 0 Faces!
Found 0 Faces!
Found 0 Faces!
Found 1 Faces!
Found 0 Faces!
Found 2 Faces!
Found 0 Faces!
Found 0 Faces!
Found 0 Faces!
Found 0 Faces!
Found 0 Faces!


In [111]:
import os

# Creamos un directorio donde guardaremos las imágenes recortadas
if not os.path.exists('scrapy/project_faces/faces_images'):
  os.makedirs('scrapy/project_faces/faces_images')

# Bucle for en el que guardamos el indice (para su posterior nombre) y las caras
face_count = 0
for i, faces in enumerate(faces_coords):
  # Bucle for para recorrer cada cara
  for face in faces:
    # Metemos cada valor de las coordenadas en una variable
    x, y, w, h = face
    # Metemos en una variable la parte de la imagen que nos interesa utilizando las coordenadas de la cara detectada  
    face_image = np.array(images[i])[y:y+h, x:x+w]
    # Creamos una imagen PIL para luego guardarla
    pil_image = Image.fromarray(face_image)
    # Guardamos la imagen recortada 
    pil_image.save(f'scrapy/project_faces/faces_images/face_{i}_{face_count}.jpg')
    face_count += 1
    

In [112]:
if os.path.exists('scrapy/project_faces/faces_images/.ipynb_checkpoints'):
  os.rmdir('scrapy/project_faces/faces_images/.ipynb_checkpoints')

# Creamos una lista con todas las imagenes
collage_images = []
for file in os.listdir('scrapy/project_faces/faces_images'):
  img = Image.open(f'scrapy/project_faces/faces_images/{file}')
  # Redimensionamos las imagenes
  img = img.resize((100, 100))
  collage_images.append(img)

# Barajamos las imagenes
import random
random.shuffle(collage_images)

# Ponemos el tamaño del ancho de nuestro collage
collage_width = 500

# Calculamos el numero de columnas y filas que necesitamos
num_images = len(collage_images)
width, height = collage_images[0].size
num_cols = int(collage_width / width)
num_rows = int(num_images / num_cols) + (num_images % num_cols > 0)

# Si nos hacen faltan mas imagenes, repite una de llas
i=0
while len(collage_images) < num_cols * num_rows:
  collage_images.append(collage_images[i])
  i+=1
# Creamos la imagen del collage
collage = Image.new('RGB', (collage_width, height * num_rows))

# Pegamos cada imagen en el collage
for i, img in enumerate(collage_images):
  x = i % num_cols
  y = int(i / num_cols)
  collage.paste(img, (x * width, y * height))

# Guardamos el collage
collage.save('scrapy/project_faces/faces_images/collage.jpg')
